In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt

class TicTacToe:
    def __init__(self):
        self.board = [" " for _ in range(9)]
        self.winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6]              # Diagonals
        ]

    def print_board(self):
        for i in range(0, 9, 3):
            print(" | ".join(self.board[i:i+3]))
            if i < 6:
                print("-" * 9)

    def available_moves(self):
        return [i for i, val in enumerate(self.board) if val == " "]

    def make_move(self, position, player):
        self.board[position] = player

    def check_winner(self, player):
        for combo in self.winning_combinations:
            if all(self.board[i] == player for i in combo):
                return True
        return False

class MCTSTreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0

def select_child(node):
    exploration_factor = 1.44  # You can experiment with different values
    return max(node.children, key=lambda c: c.wins/c.visits + exploration_factor * (node.visits**0.5 / (1 + c.visits)))

def expand_node(node):
    move = random.choice(node.state.available_moves())
    new_state = TicTacToe()
    new_state.board = node.state.board.copy()
    new_state.make_move(move, "X" if node.state.board.count("O") < node.state.board.count("X") else "O")
    new_node = MCTSTreeNode(new_state, parent=node)
    node.children.append(new_node)
    return new_node

def simulate(node):
    current_state = node.state
    while not current_state.check_winner("X") and not current_state.check_winner("O") and len(current_state.available_moves()) > 0:
        move = random.choice(current_state.available_moves())
        current_state.make_move(move, "X" if current_state.board.count("O") < current_state.board.count("X") else "O")
    return current_state.check_winner("X")

def backpropagate(node, result):
    while node is not None:
        node.visits += 1
        node.wins += result
        node = node.parent

def mcts_tic_tac_toe(iterations):
    root_state = TicTacToe()
    root_node = MCTSTreeNode(root_state)

    for _ in range(iterations):
        current_node = root_node

        # Selection
        while len(current_node.state.available_moves()) == 0 and len(current_node.children) > 0:
            current_node = select_child(current_node)

        # Expansion
        if len(current_node.state.available_moves()) > 0:
            current_node = expand_node(current_node)

        # Simulation
        simulation_result = simulate(current_node)

        # Backpropagation
        backpropagate(current_node, simulation_result)

    return max(root_node.children, key=lambda c: c.visits)

def visualize_tree(root_node):
    G = nx.Graph()
    queue = [root_node]
    while queue:
        current_node = queue.pop(0)
        G.add_node(current_node)
        if current_node.parent:
            G.add_edge(current_node.parent, current_node)
        queue.extend(current_node.children)

    pos = nx.spring_layout(G)
    labels = {node: f"Visits: {node.visits}\nWins: {node.wins}" for node in G.nodes()}
    nx.draw(G, pos, with_labels=True, labels=labels, node_size=700, node_color='skyblue', font_size=8)
    plt.show()

if __name__ == "__main__":
    iterations = 1000
    final_node = mcts_tic_tac_toe(iterations)
    visualize_tree(final_node)
